# 0. Import Package

In [1]:
import random
import scipy.stats as stats
from scipy.stats import ks_2samp
import matplotlib
import fiona
from shapely.geometry import shape
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib import gridspec
import time, datetime
import json
from urllib.parse import quote
import seaborn as sns
import pickle, csv
import numpy as np
import pandas as pd
import glob
import osmnx as ox
import igraph as ig
import itertools
from compress_pickle import dump as cdump
from compress_pickle import load as cload
import networkx as nx
from multiprocessing import Pool
import os, sys
import numpy as np
import matplotlib.ticker as ticker
import geopandas as gpd
import matplotlib.patches as mpatches
import shapely.geometry as sgeom
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import contextily as cx
import random as rand
import scipy as sp
import folium
import warnings
warnings.filterwarnings("ignore")

# 1. From Timegeo to Usertraj

In [ ]:
def userTrajectories():
    userTraj_point = {}
    userTraj_label = {}
    userTraj_zipcode = {}
    userTraj_timeSeg = {}
    userTraj_time = {}

    pointsInZipcode = pickle.load(open('data/census/pointsInZipcode.pkl', 'rb'))
    dataPath_sim = 'data/timegeo_week/'
    simFiles = [dataPath_sim + f for f in os.listdir(dataPath_sim) if os.path.isfile(os.path.join(dataPath_sim, f)) and 'txt' in f]
    simFiles = sorted(simFiles)

    # we only consider commuters here
    userCount = 0; in_list = False
    for f in simFiles:
        data = open(f, 'r')
        for line in data:
            line = line.strip().split(' ')
            if len(line) == 1:
                userCount += 1
                if userCount%10000 == 0:
                    print("user : ", userCount)
                perID = int(line[0].split('-')[0])
                otherLocations = {}
                if perID not in userTraj_point:
                    userTraj_point[perID] = []
                    userTraj_label[perID] = []
                    userTraj_zipcode[perID] = []
                    userTraj_timeSeg[perID] = []
                    userTraj_time[perID] = []
            else:
                timestep = int(line[0])
                stay_label = str(line[1])
                lon = float(line[2])
                lat = float(line[3])
                if stay_label == 'o':
                    if (lon, lat) not in otherLocations:
                        stay_label = stay_label + str(len(otherLocations))  
                        otherLocations[(lon, lat)] = stay_label
                    else:
                        stay_label = otherLocations[(lon, lat)]
                try:
                    zipcode = pointsInZipcode[(lon, lat)]
                except:
                    zipcode = ''
                # save
                userTraj_label[perID].append(stay_label)
                userTraj_point[perID].append([lon, lat])
                userTraj_zipcode[perID].append(zipcode)
                userTraj_time[perID].append(timestep)

    # save
    pickle.dump(userTraj_point, open('results/userTraj_point_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)
    pickle.dump(userTraj_label, open('results/userTraj_label_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)
    pickle.dump(userTraj_zipcode, open('results/userTraj_zipcode_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)
    pickle.dump(userTraj_time, open('results/userTraj_time_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

userTrajectories()


# 2. Calculate Route 

## 2.1: Find the Nearest Node 

In [90]:
traj_point = pickle.load(open('results/userTraj_point_all.pkl', 'rb'), encoding='bytes')
traj_time = pickle.load(open('results/userTraj_time_all.pkl', 'rb'), encoding='bytes')

In [3]:
od_time = []; user_id = []
user_keys = list(traj_point.keys())
for key in user_keys:
    od_time += traj_time[key][1:]
    user_id += [key]*(len(traj_point[key])-1)
od_time = np.array(od_time).reshape((-1,1))
user_id = np.array(user_id).reshape((-1,1))
od_lonlat = np.concatenate([np.hstack([traj_point[key][:-1],traj_point[key][1:]]).reshape((-1,4)) for key in user_keys])
od_matrix = np.concatenate([user_id,od_time,od_lonlat],axis=1)

In [4]:
point_lonlat = pd.DataFrame(np.concatenate([od_matrix[:,2:4], od_matrix[:,4:6]]))
point_lonlat = point_lonlat.drop_duplicates(subset=[0,1]).rename(columns={0:'lon',1:'lat'})
lon = point_lonlat['lon'].values; lat = point_lonlat['lat'].values
nxg = nx.relabel.convert_node_labels_to_integers(pickle.load(open('data/timeNet/network_hour_'+str(7)+'.pkl', 'rb'), encoding='bytes'))
near_point = ox.distance.nearest_nodes(nxg, lon, lat, return_dist=False)
point_lonlat['node_id'] = near_point
point_lonlat.to_csv('results/point_lonlat.csv')

In [5]:
od_point = pd.DataFrame(od_matrix).rename(columns={0:'user_id',1:'od_time',2:'o_lon',3:'o_lat',4:'d_lon',5:'d_lat'})
od_point = od_point.merge(point_lonlat, left_on=['o_lon','o_lat'], right_on=['lon','lat']).rename(columns={'node_id':'o_node_id'})
od_point = od_point.merge(point_lonlat, left_on=['d_lon','d_lat'], right_on=['lon','lat']).rename(columns={'node_id':'d_node_id'})
od_point = od_point[['user_id', 'od_time', 'o_lon', 'o_lat', 'd_lon', 'd_lat','o_node_id','d_node_id']]
od_point.to_csv('results/od_point.csv')

## 2.2: Find the Route

In [9]:
od_point = pd.read_csv('results/od_point.csv',index_col=0)
od_route = od_point.drop_duplicates(subset=['o_node_id','d_node_id','od_time'])
od_route['od_time'] = (od_route['od_time']/6).astype(int)
od_route = od_route[['od_time','o_node_id','d_node_id']]

In [17]:
for hour in range(25):
    nxg = pickle.load(open('data/timeNet/network_hour_'+str(hour)+'.pkl', 'rb'), encoding='bytes')
    G_nx = nx.relabel.convert_node_labels_to_integers(nxg)
    od_route_hour = od_route[od_route['od_time']==hour]
    s = list(od_route_hour['o_node_id'].values)
    t = list(od_route_hour['d_node_id'].values)
    od_route_detail = ox.distance.shortest_path(G_nx, s, t, weight='avg_travel_time_sec', cpus=16)
    cdump(od_route_detail, 'results/od_route_hour'+str(hour)+'.lzma', compression='lzma')

# 3. Calculate Energy Consumption

In [14]:
def parallelize_dataframe(args, func, n_cores):
    pool = Pool(n_cores)
    results = pool.map(func,args)
    pool.close()
    pool.join()
    return results

def speed2consumption(speed,miles):
    if speed < 1:
        speed = 1
    return speed_to_consumption[speed]*miles

def distance2consumption(miles):
    r = 0.342
    return miles*r

def graph2length(graph,s,t):
    return graph[s][t][0]['length']*0.000621371

def graph2time(graph,s,t):
    return graph[s][t][0]['avg_travel_time_sec']/3600

def calEnergy(hour):
    try:
        nxg = pickle.load(open('data/timeNet/network_hour_'+str(hour)+'.pkl', 'rb'), encoding='bytes'); GNSp = nx.relabel.convert_node_labels_to_integers(nxg)
    except:
        nxg = pickle.load(open('data/timeNet/network_hour_'+str(23)+'.pkl', 'rb'), encoding='bytes'); GNSp = nx.relabel.convert_node_labels_to_integers(nxg)
    od_route_hour = cload('results/od_route_hour'+str(hour)+'.lzma', compression='lzma')
    route_time = []; route_dis = []; route_energy = []

    for route in od_route_hour:
        time_total = 0; distance_total = 0; energy_total=0
        for n in range(len(route)-1):
            gdis = graph2length(GNSp,route[n],route[n+1])
            gtime = graph2time(GNSp,route[n],route[n+1])
            genergy = speed2consumption(int(gdis/gtime),gdis)
            
            time_total = time_total + gtime
            distance_total = distance_total + gdis
            energy_total = energy_total+ genergy

        route_time.append(time_total)
        route_dis.append(distance_total)
        route_energy.append(energy_total)

    pickle.dump(route_time, open('results/od_route_time_hour'+str(hour)+'.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)
    pickle.dump(route_dis, open('results/od_route_dis_hour'+str(hour)+'.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)
    pickle.dump(route_energy, open('results/od_route_energy_hour'+str(hour)+'.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

speed_to_consumption = pickle.load(open('results/speed_to_consumption.pkl', 'rb'), encoding='bytes')
results = parallelize_dataframe([hour for hour in [24]],calEnergy, n_cores=1)

In [20]:
# Get the index
od_point = pd.read_csv('results/od_point.csv',index_col=0)
od_route = od_point.drop_duplicates(subset=['o_node_id','d_node_id','od_time'])
od_route['od_time'] = (od_route['od_time']/6).astype(int)
od_route = od_route[['od_time','o_node_id','d_node_id']]
route_energy_oid = np.concatenate([od_route[od_route['od_time']==hour]['o_node_id'] for hour in range(25)])
route_energy_did = np.concatenate([od_route[od_route['od_time']==hour]['d_node_id'] for hour in range(25)])
route_energy_time = np.concatenate([od_route[od_route['od_time']==hour]['od_time'] for hour in range(25)])
# Get the energy, dis, time
route_energy_total = []
route_time_total = []
route_dis_total = []
for hour in range(25):
    route_energy = pickle.load(open('results/od_route_energy_hour'+str(hour)+'.pkl', 'rb'), encoding='bytes')
    route_energy_total += route_energy
    route_dis = pickle.load(open('results/od_route_dis_hour'+str(hour)+'.pkl', 'rb'), encoding='bytes')
    route_dis_total += route_dis
    route_time = pickle.load(open('results/od_route_time_hour'+str(hour)+'.pkl', 'rb'), encoding='bytes')
    route_time_total += route_time

In [21]:
od_route_table = pd.DataFrame({'o_node_id':route_energy_oid,'d_node_id':route_energy_did,'od_time':route_energy_time,'energy':route_energy_total,'dis':route_dis_total,'time':route_time_total})
od_route_table.to_csv('results/od_route_energy_table.csv')

# 4. Map the OD with the user

In [23]:
od_point = pd.read_csv('results/od_point.csv',index_col=0)
od_point['od_time_int'] = (od_point['od_time']/6).astype(int)
od_route_table = pd.read_csv('results/od_route_energy_table.csv',index_col=0).drop_duplicates()
od_route_results = od_point.merge(od_route_table,left_on=['od_time_int','o_node_id','d_node_id'],right_on=['od_time','o_node_id','d_node_id'])
od_route_results.to_csv('results/od_route.csv')
od_route_results_sort = od_route_results.sort_values(['user_id','od_time_x'])
od_route_results_sort.to_csv('results/od_route_results_sort.csv')

In [6]:
od_route_results_sort = pd.read_csv('results/od_route_results_sort.csv',index_col=0)
table = pd.pivot_table(od_route_results_sort, values='energy', index='user_id', aggfunc=np.sum)
traj_energy = table.to_dict('index')
pickle.dump(traj_energy, open('results/userTraj_energy_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

table = pd.pivot_table(od_route_results_sort, values='dis', index='user_id', aggfunc=np.sum)
traj_dis = table.to_dict('index')
pickle.dump(traj_dis, open('results/userTraj_rdis_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

table = pd.pivot_table(od_route_results_sort, values='time', index='user_id', aggfunc=np.sum)
traj_time = table.to_dict('index')
pickle.dump(traj_time, open('results/userTraj_rtime_all.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)